# Gaussian Process (Erf or Linear Prior)

In [1]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.kernel_ridge import KernelRidge as KR

import pandas as pd
import git

In [2]:
import warnings
warnings.simplefilter('ignore')

## Process the Datasetes

In [3]:
# return data ever since first min_cases cases
def select_region(df, fips, min_deaths=-1):
    d = df.loc[df['fips'] == fips]
    deaths = np.where(d['deaths'].values > min_deaths)[0]
    if len(deaths) == 0:
        return []
    start = deaths[0]
    d = d[start:]
    return d

In [4]:
import utils2 as utils
df = utils.get_processed_df()

<bound method NDFrame.tail of               date      county       state     fips  cases  deaths  \
0       2020-01-21   Snohomish  Washington  53061.0      1       0   
1       2020-01-22   Snohomish  Washington  53061.0      1       0   
2       2020-01-23   Snohomish  Washington  53061.0      1       0   
3       2020-01-24        Cook    Illinois  17031.0      1       0   
4       2020-01-24   Snohomish  Washington  53061.0      1       0   
...            ...         ...         ...      ...    ...     ...   
170798  2020-05-23    Sublette     Wyoming  56035.0      3       0   
170799  2020-05-23  Sweetwater     Wyoming  56037.0     25       0   
170800  2020-05-23       Teton     Wyoming  56039.0    100       1   
170801  2020-05-23       Uinta     Wyoming  56041.0     12       0   
170802  2020-05-23    Washakie     Wyoming  56043.0     20       1   

        date_processed  
0                  0.0  
1                  1.0  
2                  2.0  
3                  3.0  
4   

## Gaussian Process with Erf prior

In [5]:
import erf_model_small_changes as em
import pymc3 as pm
import arviz as az

Initialize the Erf prior as a mean function class.

In [6]:
import theano.tensor as tt

class Erf(pm.gp.mean.Mean):
    def __init__(self, fit_func, params):
        # Erf.__init__(self)
        self.fit_func = fit_func
        self.params = params

    def __call__(self, X):
        X = X.reshape(1, -1)[0]
        return em.run_model(self.fit_func, self.params, X)

In [10]:
def predict_county(county_fips):
    
    # If the number of deaths has always been 0, 
    # simply return zeroes for all percentiles
    d = select_region(df, county_fips)
    
    if len(d) == 0:
        return np.zeros((91, 9))
    
    if np.max(d['deaths'].values) == 0:
        return np.zeros((91, 9))

    with pm.Model() as gp_covid_model:
        # Lengthscale
        ρ = pm.HalfCauchy('ρ', 6)
        η = pm.HalfCauchy('η', 6)

        # Fit the Erf model
        fit_func, popt, pcov, X_train, y_train, X_pred = em.fit_erf(df, county_fips, boundary_date='2020-05-24')
        
        # Make sure predictions start from where we started training
        # X_pred = X_pred[X_pred >= X_train[0]]

        # Set the Erf prior
        M = Erf(fit_func, popt)
        K = (η**2) * pm.gp.cov.ExpQuad(1, ρ)
        
        # Maybe I should change σ depending on pcov...
        σ = pm.HalfNormal('σ', 40)

        covid_deaths_gp = pm.gp.Marginal(mean_func=M, cov_func=K)
        covid_deaths_gp.marginal_likelihood('covid_deaths', X=X_train.reshape(-1, 1), 
                               y=y_train, noise=σ)

    if len(popt) == 2:
        with gp_covid_model:
            gp_trace = pm.sample(100, tune=100, cores=2, random_seed=10)
    else:
        with gp_covid_model:
            gp_trace = pm.sample(800, tune=1500, cores=2, random_seed=35)

    with gp_covid_model:
        covid_pred_noise = covid_deaths_gp.conditional("covid_pred_noise", X_pred.reshape(-1,1), pred_noise=True)
        gp_covid_samples = pm.sample_posterior_predictive(gp_trace, vars=[covid_pred_noise], samples=500, random_seed=42)

    all_covid_samples = np.diff(gp_covid_samples['covid_pred_noise'])
    all_covid_samples = np.array(all_covid_samples)

    all_deciles = np.transpose(np.array([np.percentile(all_covid_samples, per, axis=0) for per in np.arange(10, 100, 10)]))
    all_deciles[all_deciles < 0] = 0
    
    return all_deciles

In [11]:
def predict_all_counties(out_file):
    out_dates = utils.all_output_dates()
    out_fips, all_row_starts = utils.all_output_fips('sample_submission.csv')
    num_dates, num_fips = len(out_dates), len(out_fips)
    out = np.zeros((num_dates * num_fips, 9))
    # Go through each county one by one, perform our fit, and record predictions
    for fi, fips in enumerate(out_fips):
        print('Processing FIPS', fips)
        preds = predict_county(float(fips))
        # Indices are disjointed because we're recording a single FIPS on many different dates
        out[np.arange(fi, out.shape[0], num_fips)] = preds
    # Add in the header line
    out_lines = [','.join(['id'] + ['%d' % x for x in np.arange(10, 91, 10)]) + '\n']
    # Add in all other lines one at a time
    for row_head, row in zip(all_row_starts, out):
        out_lines.append(','.join([row_head] + ['%.2f' % val for val in row]) + '\n')
    with open(out_file, 'w') as f:
        f.writelines(out_lines)
    f.close()

In [12]:
predict_all_counties('erf_model_predictions.csv')

Processing FIPS 10001


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
Sampling 2 chains, 4 divergences: 100%|██████████| 4600/4600 [00:26<00:00, 170.63draws/s]
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.
100%|██████████| 500/500 [00:04<00:00, 115.47it/s]


Processing FIPS 10003


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
100%|██████████| 500/500 [00:04<00:00, 116.24it/s]


Processing FIPS 10005


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
100%|██████████| 500/500 [00:04<00:00, 104.46it/s]


Processing FIPS 9013


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
100%|██████████| 500/500 [00:04<00:00, 111.78it/s]


Processing FIPS 9015


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
100%|██████████| 500/500 [00:04<00:00, 111.68it/s]
